In [1]:
import os
import io
from pathlib import Path
import glob
from cProfile import Profile
import pstats
import contextlib
import time

@contextlib.contextmanager
def profiler(name):
    try:
        print(f'Profiling {name}')
        pr = Profile()
        pr.enable()
        yield
    finally:
        pr.disable()
        pstats.Stats(pr).sort_stats('cumtime').print_stats(15)

def iterprofile(it, name):
    print(f'{name} Iteration Profile')
    start = time.perf_counter()
    for i, _ in enumerate(it):
        pass
    print(
        f'{i: >25} items''\n'
        f'{f"{i/(time.perf_counter()-start):0.2f}": >25} items/sec''\n'
        f'{f"{time.perf_counter()-start:0.2f}": >25} secs''\n'
    )

def profile(gen):
    with profiler(gen.__name__):
        iterprofile(gen, gen.__name__)


## Comparing Standard Library Approaches

- glob.glob
- Path.glob
- Path.iterdir
- os.walk
- os.scandir
- os.listdir

In [2]:
def globglob(root):
    yield from glob.glob('**/*', root_dir=root, recursive=True)

profile(globglob(Path.home()))

Profiling globglob
globglob Iteration Profile
                    41882 items
                 15675.13 items/sec
                     2.67 secs

         1023615 function calls (984793 primitive calls) in 2.672 seconds

   Ordered by: cumulative time
   List reduced from 100 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.051    0.051    2.672    2.672 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
    41884    0.051    0.000    2.621    0.000 /tmp/ipykernel_8719/9649885.py:1(globglob)
        1    0.053    0.053    2.570    2.570 /usr/lib/python3.10/glob.py:13(glob)
46670/41884    0.138    0.000    2.517    0.000 /usr/lib/python3.10/glob.py:53(_iglob)
    80694    0.534    0.000    1.119    0.000 /usr/lib/python3.10/posixpath.py:71(join)
     4786    0.012    0.000    0.980    0.000 /usr/lib/python3.10/glob.py:121(_glob2)
38812/4785    0.147    0.000    0.968    0.000 /usr/lib/python3.10/glob.py:167(_rlistdir)
 

In [3]:
def pathglob(root):
    root = Path(root)
    yield from root.glob('**/*')

profile(pathglob(Path.home()))

Profiling pathglob
pathglob Iteration Profile
                   818796 items
                 16571.49 items/sec
                    49.41 secs

         16455575 function calls (15385141 primitive calls) in 49.410 seconds

   Ordered by: cumulative time
   List reduced from 72 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.052    1.052   49.410   49.410 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
   818798    1.970    0.000   48.358    0.000 /tmp/ipykernel_8719/2926113733.py:1(pathglob)
   818798    1.983    0.000   46.388    0.000 /usr/lib/python3.10/pathlib.py:1023(glob)
   818798    4.480    0.000   44.405    0.000 /usr/lib/python3.10/pathlib.py:487(_select_from)
   910384    9.372    0.000   19.486    0.000 /usr/lib/python3.10/pathlib.py:438(_select_from)
  1637594    4.724    0.000   10.036    0.000 /usr/lib/python3.10/pathlib.py:668(__hash__)
1162020/91588    4.911    0.000    7.954    0.000 /usr/lib/p

In [4]:
def pathiter(root):
    root = Path(root)
    dirs = [root]
    while dirs:
        for path in dirs.pop().iterdir():
            yield path
            if path.is_dir() and not path.is_symlink():
                dirs.append(path)

profile(pathiter(Path.home()))

Profiling pathiter
pathiter Iteration Profile
                   818796 items
                 23240.60 items/sec
                    35.23 secs

         11839744 function calls in 35.231 seconds

   Ordered by: cumulative time
   List reduced from 52 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.146    1.146   35.231   35.231 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
   818798    3.567    0.000   34.085    0.000 /tmp/ipykernel_8719/4200615856.py:1(pathiter)
   818797    3.222    0.000   19.703    0.000 /usr/lib/python3.10/pathlib.py:1300(is_dir)
   910483    2.272    0.000   16.422    0.000 /usr/lib/python3.10/pathlib.py:1092(stat)
   910483    5.796    0.000   14.150    0.000 {built-in method posix.stat}
   910384    2.329    0.000    8.941    0.000 /usr/lib/python3.10/pathlib.py:1013(iterdir)
  1002070    2.480    0.000    8.695    0.000 /usr/lib/python3.10/pathlib.py:631(__fspath__)
  1002070    2.883 

In [5]:
def walk(root):
    for parent, files, dirs in os.walk(root):
        yield from (parent+d for d in dirs)
        yield from (parent+file for file in files)

profile(walk(Path.home()))

Profiling walk
walk Iteration Profile
                   818797 items
                 46886.88 items/sec
                    17.46 secs

         6539177 function calls (5468745 primitive calls) in 17.463 seconds

   Ordered by: cumulative time
   List reduced from 45 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.038    1.038   17.463   17.463 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
   818799    2.321    0.000   16.425    0.000 /tmp/ipykernel_8719/838407235.py:1(walk)
1162020/91588    6.681    0.000   12.837    0.000 /usr/lib/python3.10/os.py:345(_walk)
   910386    1.659    0.000    1.659    0.000 {built-in method builtins.next}
    91686    0.621    0.000    1.299    0.000 /usr/lib/python3.10/posixpath.py:71(join)
   818699    1.039    0.000    1.039    0.000 /tmp/ipykernel_8719/838407235.py:3(<genexpr>)
   818798    1.024    0.000    1.024    0.000 {method 'is_dir' of 'posix.DirEntry' objects}
   8187

In [6]:
def scan(root):
    dirs = [root]
    while dirs:
        with os.scandir(dirs.pop()) as directory:
            for entry in directory:
                yield entry.path
                if entry.is_dir(follow_symlinks=False):
                        dirs.append(entry.path)

profile(scan(Path.home()))

Profiling scan
scan Iteration Profile
                   818797 items
                140920.74 items/sec
                     5.81 secs

         2003998 function calls in 5.811 seconds

   Ordered by: cumulative time
   List reduced from 33 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.046    1.046    5.811    5.811 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
   818799    3.124    0.000    4.765    0.000 /tmp/ipykernel_8719/2879618333.py:1(scan)
   818798    0.973    0.000    0.973    0.000 {method 'is_dir' of 'posix.DirEntry' objects}
    91587    0.341    0.000    0.341    0.000 {built-in method posix.scandir}
    91587    0.112    0.000    0.112    0.000 {method '__exit__' of 'posix.ScandirIterator' objects}
    91587    0.108    0.000    0.108    0.000 {method 'pop' of 'list' objects}
    91586    0.107    0.000    0.107    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000  

## Recursive vs. Iterative

Quick sanity check.

In [7]:
def scan_recursive(root):
    with os.scandir(root) as directory:
        for entry in directory:
            yield entry.path
            if entry.is_dir(follow_symlinks=False):
                yield from scan_recursive(entry.path)

profile(scan_recursive(Path.home()))

Profiling scan_recursive
scan_recursive Iteration Profile
                   818798 items
                 37426.90 items/sec
                    21.88 secs

         8734484 function calls (1820827 primitive calls) in 21.877 seconds

   Ordered by: cumulative time
   List reduced from 31 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.073    1.073   21.877   21.877 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
7732457/818800   19.371    0.000   20.804    0.000 /tmp/ipykernel_8719/2039648947.py:1(scan_recursive)
   818799    0.937    0.000    0.937    0.000 {method 'is_dir' of 'posix.DirEntry' objects}
    91587    0.385    0.000    0.385    0.000 {built-in method posix.scandir}
    91587    0.111    0.000    0.111    0.000 {method '__exit__' of 'posix.ScandirIterator' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.000 /home/aidan/

## With vs. Without

How much overhead is in the with machinery and posix.ScandirIterator.\_\_exit\_\_?

In [8]:
def scan_without(root):
    dirs = [root]
    while dirs:
        for entry in os.scandir(dirs.pop()):
            yield entry.path
            if entry.is_dir(follow_symlinks=False):
                dirs.append(entry.path)

profile(scan_without(Path.home()))

Profiling scan_without
scan_without Iteration Profile
                   818797 items
                148974.22 items/sec
                     5.50 secs

         1912411 function calls in 5.496 seconds

   Ordered by: cumulative time
   List reduced from 32 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.028    1.028    5.496    5.496 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
   818799    2.961    0.000    4.469    0.000 /tmp/ipykernel_8719/3629683583.py:1(scan_without)
   818798    0.959    0.000    0.959    0.000 {method 'is_dir' of 'posix.DirEntry' objects}
    91587    0.334    0.000    0.334    0.000 {built-in method posix.scandir}
    91587    0.108    0.000    0.108    0.000 {method 'pop' of 'list' objects}
    91586    0.106    0.000    0.106    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000

## "Micro-optimizations"

- Cache methods
- Avoid the MRO on os.DirEntry.is_dir

In [9]:
def scan_micro(root):
    dirs = [root]
    pop = dirs.pop
    append = dirs.append
    scan = os.scandir
    is_dir = os.DirEntry.is_dir
    while dirs:
        for entry in scan(pop()):
            path = entry.path
            yield path
            if is_dir(entry, follow_symlinks=False):
                append(path)

profile(scan_micro(Path.home()))

Profiling scan_micro
scan_micro Iteration Profile
                   818797 items
                149512.87 items/sec
                     5.48 secs

         1912411 function calls in 5.477 seconds

   Ordered by: cumulative time
   List reduced from 32 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.033    1.033    5.477    5.477 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
   818799    2.929    0.000    4.443    0.000 /tmp/ipykernel_8719/2769856319.py:1(scan_micro)
   818798    0.961    0.000    0.961    0.000 {method 'is_dir' of 'posix.DirEntry' objects}
    91587    0.335    0.000    0.335    0.000 {built-in method posix.scandir}
    91587    0.110    0.000    0.110    0.000 {method 'pop' of 'list' objects}
    91586    0.107    0.000    0.107    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.

## Try To Catch Us Now

Let's switch to the root directory.
We should also throw some try/catch up in here so it doesn't break.

In [10]:
def scan_safe(root):
    dirs = [root]
    pop = dirs.pop
    append = dirs.append
    scan = os.scandir
    is_dir = os.DirEntry.is_dir
    while dirs:
        try:
            for entry in scan(pop()):
                path = entry.path
                yield path
                if is_dir(entry, follow_symlinks=False):
                    append(path)
        except OSError:
            continue

profile(scan_safe('/'))

Profiling scan_safe
scan_safe Iteration Profile
                  1545753 items
                154351.22 items/sec
                    10.01 secs

         3516442 function calls in 10.015 seconds

   Ordered by: cumulative time
   List reduced from 28 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.936    1.936   10.015   10.015 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
  1545755    5.404    0.000    8.078    0.000 /tmp/ipykernel_8719/1862354549.py:1(scan_safe)
  1545754    1.824    0.000    1.824    0.000 {method 'is_dir' of 'posix.DirEntry' objects}
   141628    0.514    0.000    0.514    0.000 {built-in method posix.scandir}
   141628    0.170    0.000    0.170    0.000 {method 'pop' of 'list' objects}
   141627    0.166    0.000    0.166    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.00

## Cythonize Me Cap'n

With python3.11 optimizations, cython is no longer a silver bullet here.

In [11]:
%load_ext cython

In [27]:
%%cython
# cython: profile=True
import os
def scan_cython(root: str):
    dirs: list[str] = [root]
    pop = dirs.pop
    append = dirs.append
    scan = os.scandir
    is_dir = os.DirEntry.is_dir
    while dirs:
        try:
            for entry in scan(pop()):
                path = entry.path
                yield path
                if is_dir(entry, follow_symlinks=False):
                    append(path)
        except OSError:
            continue

In [29]:
profile(scan_cython('/'))

Profiling scan_cython
scan_cython Iteration Profile
                  1545689 items
                287418.19 items/sec
                     5.38 secs

         1545741 function calls in 5.378 seconds

   Ordered by: cumulative time
   List reduced from 24 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.011    2.011    5.378    5.378 /tmp/ipykernel_8719/1959693399.py:21(iterprofile)
  1545691    3.367    0.000    3.367    0.000 /home/aidan/.cache/ipython/cython/_cython_magic_79c0fa68394ae6eb1c3f12e916a069c9.pyx:3(scan_cython)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:518(write)
        4    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:448(_schedule_flush)
        1    0.000    0.000    0.000    0.000 /home/aid

In [14]:
def make_index(path: str, root: str = '/'):
    with open(path, 'w') as file:
        for i, entry in enumerate(scan_cython(root)):
            file.write('\n'+entry)
    return i

with profiler('make_index'):
    print(make_index('fs_index'))


Profiling make_index
1542416
         3084869 function calls in 9.486 seconds

   Ordered by: cumulative time
   List reduced from 28 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    4.106    4.106    9.486    9.486 /tmp/ipykernel_8719/2017511231.py:1(make_index)
  1542418    3.404    0.000    3.404    0.000 /home/aidan/.cache/ipython/cython/_cython_magic_79c0fa68394ae6eb1c3f12e916a069c9.pyx:3(scan_cython)
  1542417    1.975    0.000    1.975    0.000 {method 'write' of '_io.TextIOWrapper' objects}
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:518(write)
        1    0.000    0.000    0.000    0.000 {method '__exit__' of '_io._IOBase' objects}
        2    0.000    0.000    0.000    0.000 /home/aidan/.l

In [40]:
%%cython
# cython: profile=True
import os
def scan_cython(root: str):
    dirs: list[str] = [root]
    pop = dirs.pop
    append = dirs.append
    scan = os.scandir
    is_dir = os.DirEntry.is_dir
    while dirs:
        try:
            for entry in scan(pop()):
                path = entry.path
                yield path
                if is_dir(entry, follow_symlinks=False):
                    append(path)
        except OSError:
            continue
def make_index_cython(path: str, root: str = '/'):
    with open(path, 'w') as file:
        for i, entry in enumerate(scan_cython(root)):
            file.write('\n'+entry)
    return i

In [41]:

with profiler('make_index'):
    print(make_index_cython('fs_index'))

Profiling make_index
1545252
         1545286 function calls in 5.714 seconds

   Ordered by: cumulative time
   List reduced from 25 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.714    5.714 {_cython_magic_8f03452f3f940590e7d7ea9fcc878e10.make_index_cython}
        1    2.361    2.361    5.714    5.714 /home/aidan/.cache/ipython/cython/_cython_magic_8f03452f3f940590e7d7ea9fcc878e10.pyx:18(make_index_cython)
  1545254    3.353    0.000    3.353    0.000 /home/aidan/.cache/ipython/cython/_cython_magic_8f03452f3f940590e7d7ea9fcc878e10.pyx:3(scan_cython)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:518(write)
        2    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:448(_schedule_flush)
        

In [42]:
%%cython
# cython: profile=True
import os
def make_index_deep_cython(path: str, root: str = '/'):
    dirs: list[str] = [root]
    pop = dirs.pop
    append = dirs.append
    scan = os.scandir
    is_dir = os.DirEntry.is_dir
    i: int = 0
    with open(path, 'w') as file:
        while dirs:
            try:
                for entry in scan(pop()):
                    path = entry.path
                    file.write('\n'+path)
                    i += 1
                    if is_dir(entry, follow_symlinks=False):
                        append(path)
            except OSError:
                continue
    return i

In [43]:
with profiler('make_index_deep_cython'):
    print(make_index_deep_cython('fs_index'))

Profiling make_index_deep_cython
1546012
         32 function calls in 1.865 seconds

   Ordered by: cumulative time
   List reduced from 24 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.865    1.865 {_cython_magic_c615842ec37d7fc7112c7bdd4541f406.make_index_deep_cython}
        1    1.865    1.865    1.865    1.865 /home/aidan/.cache/ipython/cython/_cython_magic_c615842ec37d7fc7112c7bdd4541f406.pyx:3(make_index_deep_cython)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:518(write)
        2    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:448(_schedule_flush)
        1    0.000    0.000    0.000    0.000 /home/aidan/.local/lib/python3.10/site-packages/ipykernel/iostream.py:202(schedule)
        

In [44]:
import re
def search_index(pat, path='fs_index'):
    pat = re.compile(pat)
    with open(path) as file:
        yield from (ln for ln in file if pat.match(ln))

In [45]:
%time print(len(list(search_index('.*\.py$'))))

52841
CPU times: user 438 ms, sys: 73.8 ms, total: 511 ms
Wall time: 468 ms
